In [1]:
from nnsight import LanguageModel
from nnsight.envoy import Envoy
from nnsight.util import WrapperModule
import torch
from typing import List, Tuple, Dict, Any

from transformers.utils import fx as tfx
import torch.fx as fx
from torch.fx import replace_pattern

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ff816bce810>>
Traceback (most recent call last):
  File "/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [38]:
model = LanguageModel("gpt2", device_map="auto", dispatch=True)

with model.trace("a"):
    sample_input = model.transformer.h[3].mlp.input.save()

mlp = model._model.transformer.h[3].mlp
mlp_envoy = model._envoy.transformer.h[3].mlp

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [3]:
wrapper_module = WrapperModule()
wrapper_name = 'output_wrapper'

setattr(mlp, wrapper_name, wrapper_module)

mlp

GPT2MLP(
  (c_fc): Conv1D()
  (c_proj): Conv1D()
  (act): NewGELUActivation()
  (dropout): Dropout(p=0.1, inplace=False)
  (output_wrapper): WrapperModule()
)

In [7]:
c_fc = model._model.transformer.h[3].mlp.act
c_fc_envoy = model._envoy.transformer.h[3].mlp.act

In [8]:
def custom_backend(gm: torch.fx.GraphModule, example_inputs: List[torch.Tensor]):
    print("Custom backend called with FX graph:")

    gm.graph.print_tabular()
    
    return gm.forward

torch._dynamo.reset()

opt_model = torch.compile(c_fc, backend=custom_backend)
gm = opt_model(c_fc_envoy._fake_inputs[0][0][0])

Custom backend called with FX graph:
opcode         name      target                                                   args                       kwargs
-------------  --------  -------------------------------------------------------  -------------------------  --------
placeholder    l_input_  L_input_                                                 ()                         {}
call_function  mul       <built-in function mul>                                  (0.5, l_input_)            {}
call_function  pow_1     <built-in method pow of type object at 0x7ff7efe858a0>   (l_input_, 3.0)            {}
call_function  mul_1     <built-in function mul>                                  (0.044715, pow_1)          {}
call_function  add       <built-in function add>                                  (l_input_, mul_1)          {}
call_function  mul_2     <built-in function mul>                                  (0.7978845608028654, add)  {}
call_function  tanh      <built-in method tanh of type ob

In [13]:
def my_custom_backend(gm, example_inputs):
    print("My Custom Backend:")
    print(gm.graph)
    return gm.forward

torch._dynamo.reset()

opt_model = torch.compile(mlp, backend=my_custom_backend)
gm = opt_model(mlp_envoy._fake_inputs[0][0][0])

My Custom Backend:
graph():
    %l_tensor_ : torch._subclasses.fake_tensor.FakeTensor [num_users=1] = placeholder[target=L_tensor_]
    %to : [num_users=1] = call_method[target=to](args = (%l_tensor_, 0), kwargs = {non_blocking: False})
    return (to,)
My Custom Backend:
graph():
    %l_stack0_0_0_ : torch._subclasses.fake_tensor.FakeTensor [num_users=1] = placeholder[target=L_stack0_0_0_]
    %l_module_bias : torch.nn.parameter.Parameter [num_users=1] = placeholder[target=L_module_bias]
    %l_module_weight : torch.nn.parameter.Parameter [num_users=1] = placeholder[target=L_module_weight]
    %view : [num_users=1] = call_method[target=view](args = (%l_stack0_0_0_, -1, 768), kwargs = {})
    %x : [num_users=1] = call_function[target=torch.addmm](args = (%l_module_bias, %view, %l_module_weight), kwargs = {})
    %output : [num_users=1] = call_method[target=view](args = (%x, (1, 1, 3072)), kwargs = {})
    return (output,)
My Custom Backend:
graph():
    %l_input_ : torch._subclasses.fa

In [56]:
import torch.nn as nn

class WrappedBatchNorm(nn.Module):
    def __init__(self):
        super().__init__()
        self.mod = nn.BatchNorm2d(1)
    def forward(self, x):
        return self.mod(x)

class M(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Conv2d(1, 1, 1)
        self.layer2 = nn.BatchNorm2d(1)
        self.layer3 = nn.Conv2d(1, 1, 1)
        self.nested = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.Conv2d(1, 1, 1),
        )
        self.wrapped = WrappedBatchNorm()
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.nested(x)
        x = self.wrapped(x)
        x = self.dropout(x)
        return x

def my_custom_backend(gm, example_inputs):
    print("My Custom Backend:")
    gm.graph.print_tabular()
    return gm.forward

mod = M()
mod.eval()
mod = torch.compile(mod, backend=my_custom_backend)
mod(torch.randn(1,1,4,4))

My Custom Backend:
opcode         name                target                                args                      kwargs
-------------  ------------------  ------------------------------------  ------------------------  --------
placeholder    l_x_                L_x_                                  ()                        {}
call_module    x                   L__self___layer1                      (l_x_,)                   {}
call_module    x_1                 L__self___layer2                      (x,)                      {}
call_module    x_2                 L__self___layer3                      (x_1,)                    {}
call_module    l__self___nested_0  L__self___nested_0                    (x_2,)                    {}
call_module    x_3                 L__self___nested_1                    (l__self___nested_0,)     {}
call_module    x_4                 L__self___wrapped_mod                 (x_3,)                    {}
call_function  x_5                 <function dropout 

tensor([[[[-0.4337, -0.4469, -0.4084, -0.3884],
          [-0.3759, -0.4469, -0.3876, -0.4294],
          [-0.4178, -0.4482, -0.4013, -0.4244],
          [-0.4022, -0.4329, -0.4290, -0.4147]]]],
       grad_fn=<NativeBatchNormBackward0>)

In [59]:
from typing import Optional
import torch.nn as nn
from torch import Tensor
import math 

class Conv1D(nn.Module):
    """
    1D-convolutional layer as defined by Radford et al. for OpenAI GPT (and also used in GPT-2).

    Basically works like a linear layer but the weights are transposed.

    Args:
        nf (`int`): The number of output features.
        nx (`int`): The number of input features.
    """

    def __init__(self, nf, nx):
        super().__init__()
        self.nf = nf
        self.weight = nn.Parameter(torch.empty(nx, nf))
        self.bias = nn.Parameter(torch.zeros(nf))
        nn.init.normal_(self.weight, std=0.02)

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(size_out)
        return x

class NewGELUActivation(nn.Module):
    """
    Implementation of the GELU activation function currently in Google BERT repo (identical to OpenAI GPT). Also see
    the Gaussian Error Linear Units paper: https://arxiv.org/abs/1606.08415
    """

    def forward(self, input: Tensor) -> Tensor:
        return 0.5 * input * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (input + 0.044715 * torch.pow(input, 3.0))))

class GPT2MLP(nn.Module):
    def __init__(self, intermediate_size, config):
        super().__init__()
        embed_dim = config.hidden_size
        self.c_fc = Conv1D(intermediate_size, embed_dim)
        self.c_proj = Conv1D(embed_dim, intermediate_size)
        self.act = NewGELUActivation()
        self.dropout = nn.Dropout(config.resid_pdrop)

        self.layer3 = nn.Conv2d(1, 1, 1)

    def forward(self, hidden_states: Optional[Tuple[torch.FloatTensor]]) -> torch.FloatTensor:
        hidden_states = self.c_fc(hidden_states)
        hidden_states = self.act(hidden_states)
        hidden_states = self.c_proj(hidden_states)
        hidden_states = self.dropout(hidden_states)

        # var = torch.randn(1,1,1,1)
        # hidden_states = self.layer3(var)
        
        return hidden_states

In [61]:
str(my_custom_backend)

'<function my_custom_backend at 0x7ff6eee74d60>'

In [60]:
def my_custom_backend(gm, example_inputs):
    print("My Custom Backend:")
    gm.graph.print_tabular()
    return gm.forward

mlp = GPT2MLP(64, model._model.config)
mlp.to("cuda:0")
# mlp.eval()
mod = torch.compile(mlp, backend=my_custom_backend)
mod(sample_input[0][0])

My Custom Backend:
opcode         name                     target                                                    args                                                      kwargs
-------------  -----------------------  --------------------------------------------------------  --------------------------------------------------------  --------
placeholder    l_hidden_states_         L_hidden_states_                                          ()                                                        {}
get_attr       l__self___c_fc_bias      L__self___c_fc_bias                                       ()                                                        {}
call_method    view                     view                                                      (l_hidden_states_, -1, 768)                               {}
get_attr       l__self___c_fc_weight    L__self___c_fc_weight                                     ()                                                        {}
call_function  x 

tensor([[[ 1.1716e-02,  2.4820e-02, -0.0000e+00,  1.1286e-02, -9.4275e-04,
           1.2199e-03,  2.3217e-02,  3.4676e-02, -9.3017e-03,  2.7910e-02,
           0.0000e+00, -3.1078e-02, -2.8279e-03, -1.5981e-02, -7.8414e-03,
           5.2942e-03,  2.0214e-02,  8.8420e-03,  1.5737e-02, -3.8330e-03,
           1.0344e-02, -1.1160e-02,  1.5577e-02,  1.6235e-02,  2.9588e-02,
           2.6649e-02, -7.3694e-03,  2.3291e-02,  2.2564e-02,  7.5243e-03,
          -1.0763e-02, -4.1770e-03,  1.3237e-02, -3.4895e-03, -3.4665e-03,
           1.8950e-02, -2.2814e-02, -6.8608e-03, -2.8047e-02,  7.2334e-03,
          -1.4093e-02, -2.0041e-02, -3.2132e-02, -1.0461e-02, -0.0000e+00,
           6.1334e-03, -3.6673e-03,  3.5334e-02,  3.0345e-02, -0.0000e+00,
           4.9096e-02, -5.5823e-03, -3.6563e-03,  7.1525e-03,  1.8869e-03,
          -1.6341e-02, -4.4186e-02,  2.3387e-02,  2.7945e-02,  1.1505e-02,
           1.9473e-03,  1.0681e-02, -1.2614e-02, -7.9247e-03, -0.0000e+00,
           8.4066e-04,  4